In [ ]:
import numpy as np
import tensorflow as tf
import os

In [ ]:
# Text file data converted to integer data type
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/My\ Drive/Colab\ Notebooks/AI_project


Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/AI_project


In [ ]:
!pwd

/content/gdrive/My Drive/Colab Notebooks/AI_project


In [ ]:
#Load txt file
def getRawData(string):
  lines = []

  f = open(string, "r")

  lines = f.readlines()

  return lines

In [ ]:
#turns [0,0,0,1] into 4 for directions zero default 
def getDir(line):
	if(line[0] == '1'):
		return 1
	if(line[1] == '1'):
		return 2
	if(line[3] == '1'):
		return 3
	if(line[4] == '1'):
		return 4
	return 0 

In [ ]:
def genMatricies(lines):
	x_data = []
	x_row_list = []
	y_data = []

	for x in range(0,len(lines)):
		# This is used to get rid of the \n in the txt file
		if (x % 12 == 0):
			continue
		#/the 51th line is the direction vector
		if (x % 12 == 11):
			y_data.append(getDir(lines[x]))
			
		#everything else is the matrix
		else:
			#get each char we then convert it to string for tensorflow calcs
			for sym in lines[x]:
				i = 0
				if(sym == 'X'):
					i = -1
				elif(sym == '\n'):
					continue
				elif(sym == '2'):
					i = 2
				elif(sym == '1'):
					i = 1
				x_row_list.append(i)
			#when we have 2500 chars we know we have a 50x50 make it an array and add it to list
			if(len(x_row_list) == 100):
				x_data.append(np.array(x_row_list).reshape(10,10))
				x_row_list = []
			# turn list into numpy array, turn numpy array into catagorical input
	return (np.array(x_data),tf.keras.utils.to_categorical(np.array(y_data), 4))

DATA CREATION

In [ ]:
lines = getRawData("proto10_train_gen.txt")
x1 ,y1 = genMatricies(lines)

In [ ]:
print(np.shape(x1))

(322592, 10, 10)


In [ ]:
#inf kb
lines = getRawData("proto10_train_infer.txt")
inf1 ,inf2 = genMatricies(lines)

In [ ]:
np.save('proto10_train_gen_x1.npy', x1)


In [ ]:
np.save('proto10_train_gen_y1.npy', y1)

In [ ]:
x1 = np.load("proto10_train_x1.npy")
y1 = np.load("proto10_train_y1.npy")

In [ ]:
np.save('proto10_train_inf.npy', inf1) 
#train_y1 = np.load('proto50_train_y1.npy')

In [ ]:
#train_y1 = np.load('proto50_500_train_y1.npy')
#train_y2 = np.load('proto50_train_y2.npy')
#total_y = np.append(train_y1, train_y2)
#np.save('proto50_train_total_y.npy', total_y)

TRAINING AGENT

In [ ]:
print(tf.config.list_physical_devices('GPU'))
#checkpoint_path = "/content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto50_model/proto50_train"
checkpoint_path = "/content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/proj2/CNN/"
checkpoint_dir = os.path.dirname(checkpoint_path)
EPOCHS_DEFAULT = 1000

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:

cat_array = np.concatenate((x1[:10000], inf1[:10000]), axis = 0)

y_array = y1[:10000]

print(np.shape(cat_array))
print(np.shape(y_array))

(20000, 10, 10)
(10000, 4)


In [ ]:
cat_array = np.reshape(cat_array, (-1, 10, 10, 2))


In [ ]:
print(cat_array[5])

In [ ]:
print(x1[6])
print(inf1[6])

[[ 0  0 -1 -1  0  0  0  0  0  0]
 [-1  0  0  0 -1  0  0  0  0  0]
 [ 0  0 -1 -1  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  2]]
[[ 1  2 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  0  0  0 -1 -1 -1 -1 -1 -1]
 [-1  2 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1  2 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]]


In [ ]:


def buildModel():
  global EPOCHS_DEFAULT
  # add more layers here as needed

  gridInput = tf.keras.layers.Input(shape = ( 2, 10, 10))

  flatten_image = tf.keras.layers.Flatten()( gridInput)

  dense_1 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( flatten_image )
  dense_2 = tf.keras.layers.Dense( units = 40, activation = tf.nn.relu )( dense_1 )
  dense_3 = tf.keras.layers.Dense( units = 20, activation = tf.nn.relu )( dense_2 )
  dense_4 = tf.keras.layers.Dense( units = 10, activation = tf.nn.relu )( dense_3 )

  infDec = tf.keras.layers.Dense( units = 4, activation = None )( dense_4 )

  #dense_1 = tf.keras.layers.Dense( units = 64, activation = tf.nn.relu )( flatten_image )

  #dense_2 = tf.keras.layers.Dense( units = 16, activation = tf.nn.relu )( dense_1 )

  #infDec = tf.keras.layers.Dense( units = 4, activation = None )( dense_2 )

  probabilities = tf.keras.layers.Softmax()( infDec )

  model = tf.keras.Model( inputs = gridInput, outputs = probabilities)

  model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )
  latest = tf.train.latest_checkpoint(checkpoint_dir)
 
  if(latest == None):
    return model

  model.load_weights(latest)
  EPOCHS_DEFAULT = 300

  return model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

def buildModel():
  global EPOCHS_DEFAULT

  model = Sequential()
  
  #add model layers
  model.add(Conv2D(64, kernel_size=3, activation= tf.nn.relu, input_shape=(10,10,2)))
  model.add(Conv2D(32, kernel_size=3, activation= tf.nn.relu))
  model.add(Conv2D(16, kernel_size=3, activation= tf.nn.relu))
  model.add(Flatten())
  model.add(Dense(4, activation= tf.nn.softmax))

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return model

In [ ]:
# this allows you to resume or expand upon training
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
#tf.test.is_gpu_available()

In [ ]:
model = buildModel()

In [ ]:
#model = buildModel()

#model = tf.keras.models.load_model(checkpoint_path)

#out = Concatenate()([x1, inf1])

history = model.fit( cat_array, y_array, epochs = EPOCHS_DEFAULT,
          callbacks=[cp_callback] )


Epoch 1/1000
312/313 [============================>.] - ETA: 0s - loss: 1.0265 - accuracy: 0.4552
Epoch 00001: saving model to /content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/proj2/CNN/
313/313 [==============================] - 2s 4ms/step - loss: 1.0266 - accuracy: 0.4551
Epoch 2/1000
303/313 [============================>.] - ETA: 0s - loss: 1.0097 - accuracy: 0.4607
Epoch 00002: saving model to /content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/proj2/CNN/
313/313 [==============================] - 1s 4ms/step - loss: 1.0105 - accuracy: 0.4612
Epoch 3/1000
307/313 [============================>.] - ETA: 0s - loss: 1.0063 - accuracy: 0.4713
Epoch 00003: saving model to /content/gdrive/My Drive/Colab Notebooks/AI_project/daniel_proto10_model/proj2/CNN/
313/313 [==============================] - 1s 4ms/step - loss: 1.0063 - accuracy: 0.4721
Epoch 4/1000
312/313 [============================>.] - ETA: 0s - loss: 1.0029 - accuracy: 0.4750
Ep

TESTING

In [ ]:
def getTestData(string):
  lines = []

  f = open(string, "r")

  lines = f.readlines()

  return lines

In [ ]:
test_lines = getTestData("proto10_test_gen.txt")

tx,ty = genMatricies(test_lines)

In [ ]:
print(tx[4])

[[ 0  0 -1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  1 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  2]]


In [ ]:
test_lines = getTestData("proto10_test_infer.txt")

tinf1, tinf2 = genMatricies(test_lines)

In [ ]:
np.save('proto10_test_gen_x1.npy', tx)
np.save('proto10_test_gen_y1.npy', ty)

In [ ]:
np.save('proto10_test_inf_x1.npy', tx)
np.save('proto10_test_inf_y1.npy', ty)

In [ ]:
np.save('proto10_test_infer_x1.npy', tinf1)

In [ ]:
tx = np.load('proto10_test_gen_x1.npy')
ty = np.load('proto10_test_gen_y1.npy')
tinf1 = np.load('proto10_test_infer_x1.npy')


In [ ]:
#compile model using accuracy to measure model performance
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:

test_array = np.concatenate((tx[:10000], tinf1[:10000]), axis = 0)

ty_array = ty[:10000]

In [ ]:
test_array = np.reshape(test_array, (-1, 10, 10, 2))

In [ ]:
print(np.shape(test_array))

(10000, 10, 10, 2)


In [ ]:
print(np.shape(cat_array))

(10000, 10, 10, 2)


In [ ]:

model.fit(cat_array, y_array, validation_data=(test_array, ty_array), epochs=10)

Epoch 1/10
313/313 [==============================] - 2s 6ms/step - loss: 0.2480 - accuracy: 0.8796 - val_loss: 7.2299 - val_accuracy: 0.4141
Epoch 2/10
313/313 [==============================] - 2s 6ms/step - loss: 0.2475 - accuracy: 0.8808 - val_loss: 7.2102 - val_accuracy: 0.4120
Epoch 3/10
313/313 [==============================] - 2s 6ms/step - loss: 0.2493 - accuracy: 0.8806 - val_loss: 7.3105 - val_accuracy: 0.4059
Epoch 4/10
313/313 [==============================] - 2s 6ms/step - loss: 0.2559 - accuracy: 0.8775 - val_loss: 7.2973 - val_accuracy: 0.4096
Epoch 5/10
313/313 [==============================] - 2s 8ms/step - loss: 0.2599 - accuracy: 0.8739 - val_loss: 7.0886 - val_accuracy: 0.4074
Epoch 6/10
313/313 [==============================] - 2s 6ms/step - loss: 0.2676 - accuracy: 0.8769 - val_loss: 7.0862 - val_accuracy: 0.4187
Epoch 7/10
313/313 [==============================] - 2s 5ms/step - loss: 0.2950 - accuracy: 0.8673 - val_loss: 7.2993 - val_accuracy: 0.4169
Epoch 

GEN MAZE

In [ ]:
#map constructor
from random import randint
from matplotlib import pyplot as plot


In [ ]:
#size of each grid
grid_size = 10
#dim x dim for maze
size = int(10)
#prob of wall appearance (0~1)
p_wall = 30

In [ ]:
maze= [['w' for _ in range(size)]for _ in range(size)]

In [ ]:
def createMaze():

  x_data = []
  x_row_list = []

  for row in range(size):
      for col in range(size):
          s = randint(1,100)
          p = p_wall
          if s<=p:
            maze[row][col] = 'X'
          else:
            maze[row][col] = '0'
            
  maze[0][0]= '0'
  maze[size-1][size-1] = '2'

In [ ]:
createMaze()

In [ ]:
print(maze)

[['0', '0', '0', '0', '0', 'X', 'X', '0', 'X', 'X'], ['X', 'X', 'X', '0', 'X', '0', '0', 'X', 'X', 'X'], ['0', '0', 'X', '0', '0', '0', 'X', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', 'X'], ['X', 'X', '0', 'X', '0', 'X', '0', '0', 'X', '0'], ['0', '0', 'X', 'X', 'X', 'X', 'X', '0', 'X', '0'], ['X', '0', '0', '0', '0', '0', '0', 'X', '0', 'X'], ['X', '0', 'X', '0', '0', '0', 'X', '0', '0', '0'], ['0', '0', '0', '0', 'X', '0', '0', '0', '0', '0'], ['0', 'X', '0', 'X', '0', '0', 'X', 'X', 'X', '2']]


In [ ]:
def genMaze(size):
  x_data = []
  x_row_list = []

  for row in range(size):
  #get each char we then convert it to string for tensorflow calcs
    for col in maze[row]:
      i = 0
      if(col == 'X'):
        i = -1
      elif(col == '\n'):
        continue
      elif(col == '2'):
        i = 2
      elif(col == '1'):
        i = 1
      x_row_list.append(i)
    #when we have 2500 chars we know we have a 50x50 make it an array and add it to list
  if(len(x_row_list) == 100):
    x_data.append(np.array(x_row_list).reshape(10,10))
    x_row_list = []
  # turn list into numpy array, turn numpy array into catagorical input
  return np.array(x_data)

In [ ]:
def getDir(y_arr):
  max = 0
  idx = 0
  curr = 0
  for val in y_arr:
    if val > max:
      idx = curr
    curr += 1
  
  dir = []
  for i in range(4):
    if idx != i:
      dir.append(0)
    else:
      dir.append(1)
  
  return dir

In [ ]:
new_maze = genMaze(size)

In [ ]:
print(new_maze)

[[[ 0  0  0  0  0 -1 -1  0 -1 -1]
  [-1 -1 -1  0 -1  0  0 -1 -1 -1]
  [ 0  0 -1  0  0  0 -1  0  0  0]
  [ 0  0  0  0  0  0  0  0  0 -1]
  [-1 -1  0 -1  0 -1  0  0 -1  0]
  [ 0  0 -1 -1 -1 -1 -1  0 -1  0]
  [-1  0  0  0  0  0  0 -1  0 -1]
  [-1  0 -1  0  0  0 -1  0  0  0]
  [ 0  0  0  0 -1  0  0  0  0  0]
  [ 0 -1  0 -1  0  0 -1 -1 -1  2]]]


In [ ]:
maze= [['w' for _ in range(size)]for _ in range(size)]
createMaze()
new_maze = genMaze(size)
new_maze[0][0][0] = 1
print(new_maze)

In [ ]:
print(test_array[3])

In [ ]:
y = model.predict(test_array[1])

ValueError: ignored

In [ ]:
print(test_array[1][0])

[[ 0  0  0  0  0  0  0  0  0  0]
 [-1  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]]


In [ ]:
output = getDir(y[0])
print(output)

[0, 0, 0, 1]


GEN MAZE

In [ ]:

from random import randint
from matplotlib import pyplot as plot

In [ ]:
#size of each grid
grid_size = 10
#dim x dim for maze
size = int(10)
#prob of wall appearance (0~1)
p_wall = 30

In [ ]:
maze= [['w' for _ in range(size)]for _ in range(size)]

In [ ]:
def createMaze():

  x_data = []
  x_row_list = []

  for row in range(size):
      for col in range(size):
          s = randint(1,100)
          p = p_wall
          if s<=p:
            maze[row][col] = 'X'
          else:
            maze[row][col] = '0'
            
  maze[0][0]= '0'
  maze[size-1][size-1] = '2'

In [ ]:
createMaze()

In [ ]:
print(maze)

[['0', 'X', '0', '0', 'X', 'X', 'X', 'X', '0', '0'], ['X', '0', 'X', '0', '0', '0', 'X', '0', '0', '0'], ['X', 'X', '0', '0', '0', '0', 'X', '0', '0', '0'], ['X', '0', '0', 'X', '0', 'X', '0', '0', '0', 'X'], ['0', 'X', '0', '0', 'X', '0', 'X', '0', '0', 'X'], ['0', '0', '0', 'X', '0', '0', 'X', 'X', '0', 'X'], ['X', '0', '0', 'X', '0', 'X', '0', '0', '0', '0'], ['0', 'X', 'X', '0', '0', '0', 'X', '0', 'X', '0'], ['0', '0', '0', 'X', '0', 'X', '0', 'X', 'X', '0'], ['0', '0', 'X', '0', 'X', '0', '0', 'X', '0', '2']]


In [ ]:
def genMaze(size):
  x_data = []
  x_row_list = []

  for row in range(size):
  #get each char we then convert it to string for tensorflow calcs
    for col in maze[row]:
      i = 0
      if(col == 'X'):
        i = -1
      elif(col == '\n'):
        continue
      elif(col == '2'):
        i = 2
      elif(col == '1'):
        i = 1
      x_row_list.append(i)
    #when we have 2500 chars we know we have a 50x50 make it an array and add it to list
  if(len(x_row_list) == 100):
    x_data.append(np.array(x_row_list).reshape(10,10))
    x_row_list = []
  # turn list into numpy array, turn numpy array into catagorical input
  return np.array(x_data)

In [ ]:
def getDir(y_arr):
  max = 0
  idx = 0
  curr = 0
  for val in y_arr:
    if val > max:
      idx = curr
    curr += 1
  
  dir = []
  for i in range(4):
    if idx != i:
      dir.append(0)
    else:
      dir.append(1)
  
  return dir

In [ ]:
new_maze = genMaze(size)
print(new_maze)

[[[ 0 -1  0  0 -1 -1 -1 -1  0  0]
  [-1  0 -1  0  0  0 -1  0  0  0]
  [-1 -1  0  0  0  0 -1  0  0  0]
  [-1  0  0 -1  0 -1  0  0  0 -1]
  [ 0 -1  0  0 -1  0 -1  0  0 -1]
  [ 0  0  0 -1  0  0 -1 -1  0 -1]
  [-1  0  0 -1  0 -1  0  0  0  0]
  [ 0 -1 -1  0  0  0 -1  0 -1  0]
  [ 0  0  0 -1  0 -1  0 -1 -1  0]
  [ 0  0 -1  0 -1  0  0 -1  0  2]]]


In [ ]:
maze= [['w' for _ in range(size)]for _ in range(size)]
createMaze()
new_maze = genMaze(size)
new_maze[0][0][0] = 1
print(new_maze)

[[[ 1 -1  0  0  0  0 -1  0  0 -1]
  [ 0  0  0 -1  0 -1  0  0  0  0]
  [ 0  0  0  0 -1 -1 -1  0  0  0]
  [ 0  0  0 -1  0 -1  0  0  0  0]
  [ 0 -1  0  0 -1  0  0  0  0  0]
  [-1 -1  0  0 -1  0  0  0  0  0]
  [ 0  0  0  0  0 -1  0  0 -1  0]
  [ 0  0  0  0 -1 -1  0  0  0  0]
  [-1 -1  0  0  0 -1  0  0  0  0]
  [ 0 -1  0  0  0  0  0  0 -1  2]]]
